# Single Cell Simulation Example with OBI-One Form Logic

This notebook demonstrates how to run a single cell simulation using the OBI-One form-based workflow, similar to the circuit simulation example.

In [ ]:
import obi_one as obi
from pathlib import Path
import json

# === Parameters ===
output_path = Path('.')
sim_duration = 1000.0  # ms
cell_id = 0  # Change as needed
single_cell_path = 'components/'
config_path =  f'{single_cell_path}circuit_config.json'

# === 1. Build Form (SimulationsForm) ===
sim_form = obi.SimulationsForm.empty_config()

# Info
info = obi.Info(
    campaign_name="Single Cell Simulation",
    campaign_description="Simulation of a single cell with constant current stimulus"
)
sim_form.set(info, name="info")

# Timestamps
timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=1, interval=100)
sim_form.add(timestamps, name="Timestamps")

# Stimulus
stimulus = obi.ConstantCurrentClampSomaticStimulus(
    timestamps=timestamps.ref, duration=800.0, amplitude=0.5
)
sim_form.add(stimulus, name="CurrentClampInput")

# Recording
recording = obi.SomaVoltageRecording()
sim_form.add(recording, name="SomaVoltage")

# Initialization block
with open(config_path) as f:
    cell_config = json.load(f)

# init = obi.SimulationsForm.Initialize(
#     circuit=obi.SingleCell(name="test", path=config_path),
#     simulation_length=sim_duration,
# )
init = obi.SimulationsForm.Initialize(
    circuit=obi.Circuit(name="test", path=config_path),
    simulation_length=sim_duration,
)
sim_form.set(init, name="initialize")

In [2]:
# === 2. Wrap into a Simulation ===
simulation = obi.Simulation(**sim_form.model_dump())
simulation.coordinate_output_root = output_path

# === 3. Generate simulation_config.json + node_sets.json ===
simulation.generate()  # Add db_client if needed

In [ ]:
# Remove the old compiled mod files folder
! rm -r arm64/
# flag DISABLE_REPORTINGLIB to skip SonataReportHelper.mod and SonataReport.mod from compilation.
!../../.venv/bin/nrnivmodl -incflags "-DDISABLE_REPORTINGLIB" {single_cell_path}/mechanisms

In [ ]:
# === 4. Run the simulation (BlueCelluLab backend) ===
from obi_one.scientific.simulation.execution import run

run(
    simulation_config='simulation_config.json',
    simulator='bluecellulab',
    save_nwb=False
)

## Results
The results are stored in the `output` directory. You can analyze the voltage traces and other outputs as needed.